In [6]:
## upgrade to icepyx dev breaks the query so don't use it yet, but it should be used for timing when it works
# %pip install -q --upgrade git+https://github.com/icesat2py/icepyx.git@development
# %pip install -q --upgrade icepyx

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai 2.31.2 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
dask-geopandas 0.4.3 requires dask[dataframe]>=2025.1.0, but you have dask 2024.12.1 which is incompatible.
distributed 2025.3.0 requires dask==2025.3.0, but you have dask 2024.12.1 which is incompatible.
pydantic-settings 2.8.1 requires python-dotenv>=0.21.0, but you have python-dotenv 0.20.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%load_ext autoreload
%autoreload 2

import icepyx as ipx
import geopandas as gpd

In [4]:
poly= [
      {
        "lon": 152.75574911647576,
        "lat": -81.21829026763271
      },
      {
        "lon": 157.3571761682183,
        "lat": -80.71048087173712
      },
      {
        "lon": 158.02766982432937,
        "lat": -80.55426349923613
      },
      {
        "lon": 157.09423747954733,
        "lat": -80.39105506808406
      },
      {
        "lon": 151.36217406651946,
        "lat": -80.92240258329097
      },
      {
        "lon": 152.75574911647576,
        "lat": -81.21829026763271
      }
    ]


# Extract lon and lat from the poly list
lons = [point["lon"] for point in poly]
lats = [point["lat"] for point in poly]

In [ ]:
%%time
# Build a query that points to the specific granule (no spatial filter needed)
q = ipx.Query(
    product='ATL06',
    spatial_extent=[(lon, lat) for lon, lat in zip(lons, lats)],
    date_range=['2019-12-02', '2019-12-02'],
    tracks = ['1022'],
    version='006',
) # 10220511

# Ask for S3 URLs (cloud=True) instead of HTTPS download links
gran_ids, s3_urls = q.avail_granules(ids=True, cloud=True)

# Create a cloud reader from the S3 URL and request specific vars
reader = ipx.Read(s3_urls[0])
reader.vars.append(
    beam_list=['gt2l'],
    var_list=['latitude','longitude','h_li','delta_time']
)

# Load into an xarray.Dataset (in memory, from S3)
ds = reader.load()  # default return is an xarray Dataset

In [ ]:
ds.plot.scatter(x="longitude", y="latitude", hue="h_li", vmin=-100, vmax=2000)

In [5]:
import os
import h5py
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matplotlib import pyplot as plt
data_folder = "data/"
gdf_list = []

for filename in os.listdir(data_folder):
    if filename.endswith(".h5"):
        filepath = os.path.join(data_folder, filename)
        with h5py.File(filepath, "r") as f:
            # Example for ATL06 structure, adjust as needed
            for beam in [k for k in f.keys() if k.startswith("gt")]:
                try:
                    lat = f[f"{beam}/land_ice_segments/latitude"][:]
                    lon = f[f"{beam}/land_ice_segments/longitude"][:]
                    h_li = f[f"{beam}/land_ice_segments/h_li"][:]
                    df = gpd.GeoDataFrame({
                        "h_li": h_li,
                        "beam": beam
                    }, geometry=[Point(xy) for xy in zip(lon, lat)], crs="EPSG:4326")
                    gdf_list.append(df)
                except Exception:
                    continue

if gdf_list:
    all_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs="EPSG:4326")
    all_gdf.to_parquet("data/all_segments.parquet")

/tmp/ipykernel_37145/3058036565.py:30: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  all_gdf.to_parquet("data/all_segments.parquet")
